# Разметка данных

# Пакеты

In [1]:
import pandas as pd
from enum import Enum
from datasets import Dataset, DatasetDict
%matplotlib inline

In [2]:
df = pd.read_csv("places_address_elements.csv")
df = df.fillna('')

In [3]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Адыгея,Республика,Респ,,,,,,,,,,,,
1,Башкортостан,Республика,Респ,,,,,,,,,,,,
2,Алтай,Республика,Респ,,,,,,,,,,,,
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,
4,Калмыкия,Республика,Респ,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.


# Словарь Entities

In [40]:

class ElementType(Enum):
    SEP = 0
    REGION=1
    REGION_TYPE=2
    AREA=3
    AREA_TYPE=4
    TERRITORY=5
    TERRITORY_TYPE=6
    CITY=7
    CITY_TYPE=8
    STREET=9
    STREET_TYPE=10
    OTHER = 11


# Представление адреса в виде слов-токенов и тегов NER

In [41]:
str ='Алтайский Край, Залесовский Район, Залесово Село, автодорога Мартыново-Тогул-Залесово Территория, 151-й Километр'
tokens = str.split(' ')
tokens

['Алтайский',
 'Край,',
 'Залесовский',
 'Район,',
 'Залесово',
 'Село,',
 'автодорога',
 'Мартыново-Тогул-Залесово',
 'Территория,',
 '151-й',
 'Километр']

In [42]:
class TypedWord:
    word = ''
    type = ElementType.OTHER
    def __init__(self, word:str, type:ElementType):
        self.word = word
        self.type = type
    
    def __repr__(self):
        return self.word + ":" + self.type.name

### Функция конвертации набора адресо-образующих элементов в токены и NER-теги

In [47]:
def tokens_with_tag(input , elementType :ElementType  ) -> list:
    words = list(filter(('').__ne__, input.split(' ')))
    res = list()
    for word in words:
        res.append(TypedWord(word, elementType))
    return res


def convert_address_elements(elements: dict):
    return (tokens_with_tag(elements['region_name'], ElementType.REGION) +
            tokens_with_tag(elements['region_type_name'], ElementType.REGION_TYPE) +
            tokens_with_tag(",", ElementType.SEP) +
            tokens_with_tag(elements['area_name'], ElementType.AREA) +
            tokens_with_tag(elements['area_type_name'], ElementType.AREA_TYPE) +
            tokens_with_tag(",", ElementType.SEP) +
            tokens_with_tag(elements['city_name'], ElementType.CITY) +
            tokens_with_tag(elements['city_type_name'], ElementType.CITY_TYPE) +
            tokens_with_tag(",", ElementType.SEP) +
            tokens_with_tag(elements['territory_name'], ElementType.TERRITORY) +
            tokens_with_tag(elements['territory_type_name'], ElementType.TERRITORY_TYPE) +
            tokens_with_tag(",", ElementType.SEP) +
            tokens_with_tag(elements['street_name'], ElementType.STREET) +
            tokens_with_tag(elements['street_type_name'], ElementType.STREET_TYPE)
            )

elements = df.sample().to_dict('records')
convert_address_elements(elements[0])

[Калмыкия:REGION,
 Республика:REGION_TYPE,
 ,:SEP,
 ,:SEP,
 Элиста:CITY,
 Город:CITY_TYPE,
 ,:SEP,
 ,:SEP,
 Сян:STREET,
 Бяядл:STREET,
 Улица:STREET_TYPE]

In [26]:
elements = df.sample().to_dict('records')

In [27]:
convert_address_elements(elements[0])

KeyError: 'street_type_name_'

In [69]:
def convert_address_elements_df(df:pd.DataFrame):
    return [convert_address_elements(item) for item in df.to_dict('records')]

convert_address_elements_df(r)


[{'tokens': ['Алтайский',
   'край',
   'Залесовский',
   'р-н',
   'Залесово',
   'с',
   'Полевая',
   'ул'],
  'ner_tags': [0, 1, 2, 3, 6, 7, 8, 9]}]

## Appply to dataframe

In [70]:
s = df.apply(convert_address_elements, axis=1)

In [71]:
new_df = pd.DataFrame(list(s))

In [72]:
new_df

,tokens,ner_tags
0,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
1,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 4, 5, 5, 8, 9]"
2,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
3,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
4,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
...,...,...
612825,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612826,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612827,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612828,"[Еврейская, Аобл, Биробиджан, г, СОТ, Багульни...","[0, 1, 6, 7, 4, 5, 5, 8, 9]"


## В формате Datasets

In [73]:
dataset = Dataset.from_pandas(new_df)

In [74]:
train_testvalid = dataset.train_test_split(test_size=0.33, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})

In [75]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 410596
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
})

In [76]:
dataset.save_to_disk('address_elements')

Saving the dataset (0/1 shards):   0%|          | 0/410596 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]